# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

2025-02-09 07:38:31.861193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739104711.929294    8408 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739104711.949532    8408 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 07:38:32.102255: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
target_y = loans_df['credit_ranking']

# Display a sample of y
target_y.sample(4)

446    0
423    1
857    1
8      1
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking
X_values = loans_df.drop(columns = ['credit_ranking'])

# Review the features DataFrame
X_values.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
x_train, x_test, y_train, y_test = train_test_split(X_values, target_y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaly = StandardScaler()

# Fit the scaler to the features training dataset
scaly.fit(x_train)

# Fit the scaler to the features training dataset
# The above is understood to be a typo and "Transform the x data to scaled data" is the goal. 
x_train_scaled = scaly.transform(x_train)
x_test_scaled = scaly.transform(x_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the the number of inputs (features) to the model
num_feat = len(x_train.columns)

# Review the number of features
num_feat

11

In [12]:
# Define the number of hidden nodes for the first hidden layer
first_hid_node = 64

# Define the number of hidden nodes for the second hidden layer
second_hid_node = 32

# Define the number of neurons in the output layer
output_node = 1

In [16]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(first_hid_node, activation='relu', input_shape = (num_feat,)))

# Add the second hidden layer
model.add(Dense(second_hid_node, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(output_node, activation='sigmoid'))

/home/ellz/anaconda3/envs/wave-rider/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-09 07:47:21.158272: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [17]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [18]:
# Compile the Sequential model
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Fit the model using 50 epochs and the training data
model.fit(x_train_scaled, y_train, epochs = 50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7000 - loss: 0.5999 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7298 - loss: 0.5256 
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - accuracy: 0.7586 - loss: 0.5069
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7456 - loss: 0.4940 
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - accuracy: 0.7923 - loss: 0.4603
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.8038 - loss: 0.4534
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7816 - loss: 0.4705 
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.7864 - loss: 0.4629
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.7990 - loss: 0.4503
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step - accuracy: 0.8013 - loss: 0.4412
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7856 - loss: 0.4479 
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - ac

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [21]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model.evaluate(x_test_scaled, y_test)

# Display the model loss and accuracy results


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7887 - loss: 0.5189  


[0.5516303777694702, 0.7549999952316284]

### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [22]:
# Set the model's file path
path = Path("student_loans.keras")

# Export your model to a keras file
model.save(path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [24]:
# Set the model's file path
path = Path("student_loans.keras")

# Load the model to a new object
model = tf.keras.models.load_model(path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [31]:
# Make predictions with the test data
predictions = pd.DataFrame(model(x_test_scaled).numpy())
# Display a sample of the predictions
predictions.sample(5)

,0
387,0.626111
132,0.959289
71,0.984007
314,0.832022
95,0.303323


In [35]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions = predictions.round(0)
predictions

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,1.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [37]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.75      0.72      0.73       188
           1       0.76      0.79      0.77       212

    accuracy                           0.76       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.76      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
The model is fitted to a student's payment history, location, stem degree/major, gpa ranking, the success of their alumni at the given school, major, time to complete their degree, finance workshop score, cohort ranking, total loan score, and financial aid score. In accordance with the above accuracy scores, this aligns decently with the resultant credit ranking which presumably is the measure of success a student had with their loan (as a lower score may signify they've had difficulties paying off the loan while a higher one implies the opposite). 
This data makes sense and is appropriate because it points to their life habits, like how they perform in their academic setting which might give us an idea of how they will handle costs related to their academics as well as their financial history and financial literacy. So sorting students into which loans work the best for these circumstances will probably lead to better fits.  

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
In this case context-based filtering would be ideal. The context of the student such as their gpa, life habits, and financial literacy as outlinded above should be used to filter the students into appropriate loan categories as this would most likely relate to their satisfaction with the type of loan they receive.  

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
The students might have multiple loans which may affect the type of loan they would prefer, they might be international which may affect how well the loans are recommended to them as international students are concerned with other aspects as well as loans (such as visas), students with lower GPAs or other aspects that they may not wish to disclose to a form might be underrepresented in the users and therefore the lower GPA fits may not be as good. When collecting data, we should add in little bits like "no judgement zone" etc to coax otherwise nervous users, other loans should be taken into account as should international status. In addition credit ranking might also be a tricky metric to use as a target to suss out which features are ideal. We might want to switch to a score which measures the relative repayment when considering the student's financial background. 
